In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_excel('./data/WTI.xlsx', 'Spot', index_col='Date')
df2 = pd.read_excel('./data/WTI.xlsx', 'Front', index_col='Date')
df3 = pd.read_excel('./data/WTI.xlsx', 'Second', index_col='Date')

In [3]:
df1.head(3)

,Spot
Date,
2017-12-28,59.84
2017-12-27,59.64
2017-12-26,59.97


In [4]:
df2.head(3)

,Front
Date,
2017-12-28,59.84
2017-12-27,59.64
2017-12-26,59.97


In [5]:
df3.head(3)

,Second
Date,
2017-12-28,59.87
2017-12-27,59.69
2017-12-26,60.00


In [6]:
df = pd.concat([df1, df2, df3], axis=1)
df.head()

,Spot,Front,Second
Date,,,
2017-12-28,59.84,59.84,59.87
2017-12-27,59.64,59.64,59.69
2017-12-26,59.97,59.97,60.00
2017-12-22,58.42,58.47,58.54
2017-12-21,58.33,58.36,58.40


In [7]:
df = df.fillna(method='ffill')

In [8]:
df_e = pd.read_excel('./data/WTI.xlsx', 'Expiry')
df_e[120:125]

,Ticker,Month,Last Trade,Last Price
120,CLG10,2017-02-10,01/20/10,77.62
121,CLH10,2017-03-10,02/22/10,80.16
122,CLJ10,2017-04-10,03/22/10,81.25
123,CLK10,2017-05-10,04/20/10,83.45
124,CLM10,2017-06-10,05/20/10,68.01


In [9]:
# 만기일 추출
df_e['Last Trade'].head()

0    01/20/00
1    02/22/00
2    03/21/00
3    04/19/00
4    05/22/00
Name: Last Trade, dtype: object

In [10]:
# 만기일을 날짜 형식으로 변환
pd.to_datetime(df_e['Last Trade']).head()

0   2000-01-20
1   2000-02-22
2   2000-03-21
3   2000-04-19
4   2000-05-22
Name: Last Trade, dtype: datetime64[ns]

In [11]:
# 만기일 정보를 날짜 형태로 바꾸고 expiry에 저장
expiry = pd.to_datetime(df_e['Last Trade'])
expiry.head()

0   2000-01-20
1   2000-02-22
2   2000-03-21
3   2000-04-19
4   2000-05-22
Name: Last Trade, dtype: datetime64[ns]

In [12]:
# expiry 를 리스트 형태로 바꾸고 exp에 저장
exp = expiry.tolist()
exp

[Timestamp('2000-01-20 00:00:00'),
 Timestamp('2000-02-22 00:00:00'),
 Timestamp('2000-03-21 00:00:00'),
 Timestamp('2000-04-19 00:00:00'),
 Timestamp('2000-05-22 00:00:00'),
 Timestamp('2000-06-20 00:00:00'),
 Timestamp('2000-07-20 00:00:00'),
 Timestamp('2000-08-22 00:00:00'),
 Timestamp('2000-09-20 00:00:00'),
 Timestamp('2000-10-20 00:00:00'),
 Timestamp('2000-11-17 00:00:00'),
 Timestamp('2000-12-19 00:00:00'),
 Timestamp('2001-01-22 00:00:00'),
 Timestamp('2001-02-20 00:00:00'),
 Timestamp('2001-03-20 00:00:00'),
 Timestamp('2001-04-20 00:00:00'),
 Timestamp('2001-05-22 00:00:00'),
 Timestamp('2001-06-20 00:00:00'),
 Timestamp('2001-07-20 00:00:00'),
 Timestamp('2001-08-21 00:00:00'),
 Timestamp('2001-09-20 00:00:00'),
 Timestamp('2001-10-22 00:00:00'),
 Timestamp('2001-11-19 00:00:00'),
 Timestamp('2001-12-18 00:00:00'),
 Timestamp('2002-01-22 00:00:00'),
 Timestamp('2002-02-20 00:00:00'),
 Timestamp('2002-03-20 00:00:00'),
 Timestamp('2002-04-22 00:00:00'),
 Timestamp('2002-05-

In [13]:
# 데이터프레임에 Expiry 컬럼 생성
df['Expiry'] = ''
df.head()

,Spot,Front,Second,Expiry
Date,,,,
2017-12-28,59.84,59.84,59.87,
2017-12-27,59.64,59.64,59.69,
2017-12-26,59.97,59.97,60.00,
2017-12-22,58.42,58.47,58.54,
2017-12-21,58.33,58.36,58.40,


In [14]:
# Trading day 중 만기일 찾아서 표시하기
for d in df.index:
    if d in exp:
        df.loc[d, 'Expiry'] = 1
df[25:30] 

,Spot,Front,Second,Expiry
Date,,,,
2017-11-21,56.72,56.83,56.92,
2017-11-20,56.09,56.09,56.42,1
2017-11-17,56.55,56.55,56.71,
2017-11-16,55.14,55.14,55.35,
2017-11-15,55.33,55.33,55.52,


In [15]:
# 현물에 투자했다면
# 손익 = 현재가격 - 초기가격
price_now = float(df.loc['2017-12-28', 'Spot'])
price_before = float(df.loc['2010-01-04', 'Spot'])
profit = (price_now - price_before) * 1000
print('손익 (현재가격 - 초기가격) : ', '{:,.2f}'.format(profit))

손익 (현재가격 - 초기가격) :  -21,670.00


In [16]:
# 선물에 투자했다면
# 손익 = 현재가격 - 초기가격 - 롤오버 비용
roll_over_cost_acc = 0
for d in df.index:
    if (df.loc[d, 'Expiry'] == 1):
        # 롤오버 비용 = 차근월물 가격 - 최근월물 가격
        roll_over_cost = df.loc[d, 'Second'] - df.loc[d, 'Front']
        roll_over_cost_acc = roll_over_cost_acc + roll_over_cost
        
print('누적 롤오버 비용 :', '{:,.2f}'.format(roll_over_cost_acc))

누적 롤오버 비용 : 40.68


In [17]:
price_now = float(df.loc['2017-12-28', 'Front'])
price_before = float(df.loc['2010-01-04', 'Front'])
profit = (price_now - price_before - roll_over_cost_acc) * 1000
print('누적 손익', '{:,.2f}'.format(profit))

누적 손익 -62,350.00
